<a href="https://colab.research.google.com/github/Carlo-Cascini/Carlo-Cascini/blob/main/test_pci_etfs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install_github("matthewclegg/partialAR")
install_github("matthewclegg/partialCI")

misc3d (NA -> 0.9-1) [CRAN]
plot3D (NA -> 1.4.1) [CRAN]
KFAS   (NA -> 1.5.1) [CRAN]


Installing 3 packages: misc3d, plot3D, KFAS

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/Rtmpo6UdMl/remotes2c67e971c35/matthewclegg-partialAR-67aa0b9/DESCRIPTION’ ... OK
* preparing ‘partialAR’:
* checking DESCRIPTION meta-information ... OK
* cleaning src
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘partialAR_1.0.13.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)





── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/Rtmpo6UdMl/remotes2c615d15f23/matthewclegg-partialCI-c2561de/DESCRIPTION’ ... OK
* preparing ‘partialCI’:
* checking DESCRIPTION meta-information ... OK
* cleaning src
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building ‘partialCI_1.1.1.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(partialAR)
library(partialCI)

Warning message:
“replacing previous import ‘zoo::yearmon’ by ‘data.table::yearmon’ when loading ‘partialAR’”
Warning message:
“replacing previous import ‘zoo::yearqtr’ by ‘data.table::yearqtr’ when loading ‘partialAR’”
Warning message:
“no DISPLAY variable so Tk is not available”


In [ ]:
# install.packages("devtools")
# install.packages("quantmod")
# install.packages("xts")
# install.packages("zoo")
# install.packages("RcppRoll")
library(devtools)
library(quantmod)
library(xts)
library(zoo)
stock_a <- "EWA"
stock_b <- "EWK"

# Retrieve historical data for the stocks
getSymbols(stock_a, src = "yahoo", from = "2010-01-01", to = "2012-12-31")
a_prices <- Ad(get(stock_a))

getSymbols(stock_b, src = "yahoo", from = "2010-01-01", to = "2012-12-31")
b_prices <- Ad(get(stock_b))

In [ ]:
# Merge the adjusted closing prices
prices <- merge(a_prices, b_prices, all = FALSE)
prices_xts <- xts(prices, order.by = index(prices))
fit<- fit.pci(prices[,1],prices[,2])
print(fit)

In [ ]:
# Assuming fit is your fitted model object
sigmak <- fit$sigma_M
sigmaz <- fit$sigma_R
rho <- fit$rho

rsq_MR <- 2 * sigmak^2 / (2 * sigmak^2 + (1 + rho) * sigmaz^2)

# Check conditions
if (rsq_MR > 0.5 && rho > 0.5 && rho < 1) {
  cat("Condition met:\n")
} else {
  cat("Condition not met:\n")
}

Condition met:


In [ ]:
# Define a list of stock tickers
stock_tickers <- c("EWA", "EWK", "EWO", "EWC", "EWQ", "EWG", "EWH", "EWI", "EWJ", "EWM", "EWW",
                    "EWN", "EWS", "EWP", "EWD", "EWL", "EWY", "EZU", "EWU", "EWZ", "EWT", "SPY")

# Generate unique pairs of stock tickers
stock_pairs <- combn(stock_namelist, 2, simplify = FALSE)

# Function to calculate rsq_MR and check conditions
calculate_rsq_MR <- function(stock_a, stock_b) {
  tryCatch({
    # Ensure stock_a is alphabetically before stock_b
    if (stock_a > stock_b) {
      temp <- stock_a
      stock_a <- stock_b
      stock_b <- temp
    }

    # Retrieve historical data for the stocks
    getSymbols(stock_a, src = "yahoo", from = "2010-01-01", to = "2012-12-31")
    a_prices <- Ad(get(stock_a))

    getSymbols(stock_b, src = "yahoo", from = "2010-01-01", to = "2012-12-31")
    b_prices <- Ad(get(stock_b))

    # Merge the adjusted closing prices
    prices <- merge(a_prices, b_prices, all = FALSE)
    prices_xts <- xts(prices, order.by = index(prices))

    # Fit the PCI model
    fit <- fit.pci(prices[,1], prices[,2])

    # Calculate rsq_MR
    sigmak <- fit$sigma_M
    sigmaz <- fit$sigma_R
    rho <- fit$rho
    rsq_MR <- 2 * sigmak^2 / (2 * sigmak^2 + (1 + rho) * sigmaz^2)

    # Check conditions
    if (rsq_MR > 0.5 && rho > 0.5 && rho < 1) {
      return(list(stock_a = stock_a, stock_b = stock_b, rsq_MR = rsq_MR, rho = rho))
    } else {
      return(NULL)
    }
  }, error = function(e) {
    return(NULL)  # Handle errors gracefully
  })
}

# Iterate over each stock pair and check conditions
results <- lapply(stock_pairs, function(pair) {
  calculate_rsq_MR(pair[1], pair[2])
})

# Filter out NULL results (where conditions were not met)
results <- Filter(function(x) !is.null(x), results)

# Print results
if (length(results) > 0) {
  cat("Conditions met for the following stock pairs:\n")
  for (result in results) {
    cat("Stock Pair:", result$stock_a, "-", result$stock_b, "\n")
    cat("R^2[MR] =", result$rsq_MR, "\n")
    cat("rho =", result$rho, "\n\n")
  }
} else {
  cat("No stock pairs met the conditions.\n")
}

Conditions met for the following stock pairs:
Stock Pair: EWA - EWK 
R^2[MR] = 0.9114087 
rho = 0.9572616 

Stock Pair: EWA - EWQ 
R^2[MR] = 0.7999312 
rho = 0.9410857 

Stock Pair: EWA - EWG 
R^2[MR] = 0.9547806 
rho = 0.9663774 

Stock Pair: EWA - EWH 
R^2[MR] = 0.7365957 
rho = 0.9420171 

Stock Pair: EWA - EWJ 
R^2[MR] = 0.854506 
rho = 0.982935 

Stock Pair: EWA - EWW 
R^2[MR] = 0.8358902 
rho = 0.9618335 

Stock Pair: EWA - EWN 
R^2[MR] = 0.8671824 
rho = 0.9422596 

Stock Pair: EWA - EWS 
R^2[MR] = 0.892299 
rho = 0.9877891 

Stock Pair: EWA - EWD 
R^2[MR] = 0.7837451 
rho = 0.9367406 

Stock Pair: EWA - EWL 
R^2[MR] = 0.991424 
rho = 0.9439379 

Stock Pair: EWA - EWY 
R^2[MR] = 0.9147308 
rho = 0.9653772 

Stock Pair: EWA - EZU 
R^2[MR] = 0.7957691 
rho = 0.9473461 

Stock Pair: EWA - EWU 
R^2[MR] = 0.9430406 
rho = 0.9216462 

Stock Pair: EWA - EWT 
R^2[MR] = 0.9321089 
rho = 0.9710771 

Stock Pair: EWC - EWK 
R^2[MR] = 1 
rho = 0.9968502 

Stock Pair: EWG - EWK 
R^2[MR] = 1 
